In [ ]:
###########import packages##########
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn import svm
from sklearn import neighbors
from sklearn import tree
from sklearn.linear_model import BayesianRidge
from sklearn.tree import ExtraTreeRegressor
from sklearn.tree import ExtraTreeClassifier
from sklearn import linear_model
import lightgbm
import catboost
import xgboost
from sklearn.metrics import r2_score
%matplotlib
###########wrapping root mean square error for later calls##########
def compute_mae_mse_rmse(target,prediction):
    error = []
    for i in range(len(target)):
        error.append(target[i] - prediction[i])
    squaredError = []
    absError = []
    for val in error:
        squaredError.append(val * val)  # target-prediction之差平方
        absError.append(abs(val))  # 误差绝对值
    mae=sum(absError)/len(absError)  # 平均绝对误差MAE
    mse=sum(squaredError)/len(squaredError)  # 均方误差MSE
    RMSE=sum(absError)/len(absError)
    R2=r2_score(target,prediction)
    return mae,mse,RMSE,R2
###########loading data##########
fdata=pd.read_csv('database_filled.csv',encoding="gbk")
raw_data=fdata.loc[:,[                      
                      'Fe Precursor Ratio',#14
                      'Zn Precursor Ratio',#15
                      '2-Methylimidazole Precursor Ratio',#16
                      'Additive (Other) Precursor Ratio',#17
                      'ZIF Preparation Temperature (℃)',#18
                      'ZIF Preparation Time (h)',#19
                      'One-Pot Method',#20
                      'Post-Adsorb Method',#21
                      'Carbon-Derivative Adsorb Method',#22
                      'Pyrolysis Temperature (℃)',#23
                      'Pyrolysis Time (h)',#24
                      'Temperature Rising Rate (℃ min-1)',#25
                      'Seconde Pyrolysis',#26
                      'Acid Wash',#13
                      'Content of Pyridinic Species (at. %)'#the regression target#14
                        ]]
###########train test splitting##########
raw_param=raw_data.iloc[:,0:14]
raw_power=raw_data.iloc[:,14]

In [ ]:
def gridsearch(model,param,algorithm_name):
    print('start')
    grid = GridSearchCV(model,param_grid=param,cv=5)
    grid.fit(X_train,y_train)
    best_model=grid.best_estimator_
    result = best_model.predict(X_test)
    x_prediction_07=result
    y_real_07=y_test_values
    x_prediction_07_series=pd.Series(x_prediction_07)
    y_real_07_series=pd.Series(y_real_07)
    ###########evaluating the regression quality##########
    corr_ann = round(x_prediction_07_series.corr(y_real_07_series), 5)
    error_val= compute_mae_mse_rmse(x_prediction_07,y_real_07)
    rank = [index for index, value in sorted(list(enumerate(best_model.feature_importances_)), key=lambda x:x[1],reverse=True)]
    ###########generating a figure##########
    print('Best Regressor:',grid.best_params_,'Best Score:', grid.best_score_)
    print(error_val)
    print(corr_ann)
    x_y_x=np.arange(0,0.1,0.01)
    x_y_y=np.arange(0,0.1,0.01)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.scatter(x_prediction_07,y_real_07,color='red',label=algorithm_name)
    ax.plot(x_y_x,x_y_y)
    plt.legend()
    plt.xlabel(u"Predicted_Content_of_Pyridinic_Species_(at. %)")
    plt.ylabel(u"Real_Content_of_Pyridinic_Species_(at. %)")
    plt.savefig('%s scatters.png' %algorithm_name)
    fig2 = plt.figure()
    name_list=[   
                      'Fe pre ratio',#0
                      'Zn pre ratio',#1
                      '2mem pre ratio',#2
                      'additive pre ratio',#3
                      'ZIF preparation temperature',#4
                      'ZIF preparation time',#5
                      'one-pot',#6
                      'second-adsorb',#7
                      'carbon-adsorb',#8
                      'Pyrolysis Temperature in inert atomosphere',#9
                      'Pyrolysis Time inert',#10
                      'Rising Rate',#11
                      'second pyrolysis',#12
                      'Acid Wash (1 for yes 0 for no)'
            ]
    plt.bar(range(len(best_model.feature_importances_)), best_model.feature_importances_,color='rgb',tick_label=name_list)
    plt.xticks(rotation=90)
    plt.savefig('%s importance.png' %algorithm_name)
    print('finished')

In [ ]:
seed=392
X_train, X_test, y_train, y_test = train_test_split(raw_param, raw_power, test_size=.15,random_state=seed)    
y_test_values=y_test.values.astype(np.float32)

##########LGBM gridsearch CV for best hyperparameter##########
model_LightGBMRegressor=lightgbm.LGBMRegressor(random_state=1,verbose=-1)
param_light = {
'boosting_type':['gbdt','rf'],
'learning_rate':[0.01,0.02,0.05,0.1,0.2,1],
'n_estimators':[100,200,400,800],
'subsample':[0.5,0.6,0.7,0.8,0.9,1],
'reg_alpha':[0,0.001,0.01],
'reg_lambda':[0,0.001,0.01]
}
gridsearch(model_LightGBMRegressor,param_light,'LightGBM')

##########XGBoost gridsearch CV for best hyperparameter##########
model_XGBRegressor=xgboost.XGBRegressor(objective='reg:squarederror',random_state=1,verbose=0)
param_xg = {
'booster':['gbtree'],
'learning_rate':[0.01,0.02,0.05,0.1,0.2,1],
'n_estimators':[100,200,400,800],
'max_depth':[5,7,9,11],
'subsample':[0.5,0.6,0.7,0.8,0.9,1],
'reg_alpha':[0,0.001,0.01],
'reg_lambda':[0,0.001,0.01]
}
gridsearch(model_XGBRegressor,param_xg,'XGBoost')

##########CatBoost gridsearch CV for best hyperparameter##########
model_CatRegressor=catboost.CatBoostRegressor(random_state=1,verbose=0)
param_cat = {
 'learning_rate':[0.01,0.05,0.1],
 'n_estimators':[200,400],
 'max_depth':[5,8,11],
 'subsample':[0.5,0.6,0.7,0.8,0.9,1]
}
gridsearch(model_CatRegressor,param_cat,'CatBoost')


###########GradientBoost gridsearch CV for best hyperparameter##########
model_GradientBoostingRegressor = ensemble.GradientBoostingRegressor(random_state=1)
###########defining the parameters dictionary##########
param_GB = {
        'learning_rate':[0.01,0.05,0.1],
        'n_estimators':[200,400],
        'criterion':['friedman_mse','mae','mse'],
         'max_features':['auto','sqrt','log2'],
         'loss':['ls', 'lad', 'huber', 'quantile']
}
gridsearch(model_GradientBoostingRegressor,param_GB,'GradientBoost')

###########RandomForest gridsearch CV for best hyperparameter##########
model_RandomForestRegressor = ensemble.RandomForestRegressor(random_state=1)
###########defining the parameters dictionary##########
param_RF = {
            'n_estimators':[100,200,400,800],
    'criterion':['mse','mae'],
    'max_features':['auto','sqrt','log2']
}
gridsearch(model_RandomForestRegressor,param_RF,'Random Forest')


###########Extra Tree gridsearch CV for best hyperparameter##########
model_ExtraTreeRegressor = ExtraTreeRegressor(random_state=1)
param_ET = {
               'max_depth':[5,7,9,11],
       'max_features':['auto','sqrt','log2'],
       'criterion' : ["mse", "friedman_mse", "mae"],
       'splitter' : [ "best",'random']
}
gridsearch(model_ExtraTreeRegressor,param_ET,'Extra Tree')


###########Decision Tree gridsearch CV for best hyperparameter##########
model_DecisionTreeRegressor = tree.DecisionTreeRegressor(random_state=1)
param_DT = {
               'max_depth':[5,7,9,11],
       'max_features':['auto','sqrt','log2'],
       'criterion' : ["mse", "friedman_mse", "mae"],
       'splitter' : [ "best",'random']
}
gridsearch(model_DecisionTreeRegressor,param_DT,'Decision Tree')


###########AdaBoost gridsearch CV for best hyperparameter##########
model_AdaBoostRegressor = ensemble.AdaBoostRegressor(random_state=1)
param_Ada = {
            'n_estimators':[50,100,200,400,800],
    'learning_rate':[0.01,0.02,0.05,0.1,0.2,1],
    'loss':['linear', 'square', 'exponential']
}
gridsearch(model_AdaBoostRegressor,param_Ada,'AdaBoost')